In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
import sklearn
import numpy
import scipy
import plotly
import bs4 as bs
import urllib.request
import boto3
import pandas as pd

In [0]:
import os
# To work with Amazon S3 storage, set the following variables using your AWS Access Key and Secret Key
# Set the Region to where your files are stored in S3.
# access_key =
# secret_key =
# Set the environment variables so boto3 can pick them up later
os.environ['AWS_ACCESS_KEY_ID'] = access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
encoded_secret_key = secret_key.replace("/", "%2F")
aws_region = "us-east-2"
# Update the Spark options to work with our AWS Credentials
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region +
".amazonaws.com")

In [0]:
### Cleaning JSON File #1: yelp_academic_dataset_business.json ###

In [0]:
yelp_business_file = 's3a://yelp-reviews-jb/landing/yelp_academic_dataset_business.json'
business_sdf = spark.read.json(yelp_business_file)

In [0]:
# New dataframe with no null values

from pyspark.sql.functions import col

business_sdf = business_sdf.dropna()

In [0]:
# Dropping Unnecessary Columns

columns_to_drop = ['postal_code', 'latitude', 'longitude']

business_sdf = business_sdf.drop(*columns_to_drop)

business_sdf.show()

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+--------------------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|                name|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+--------------------+------------+-----+-----+
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|       Affton|{8:0-18:30, 0:0-0...|      1|       The UPS Store|          15|  3.0|   MO|
|5255 E Broadway Blvd|{null, null, null...|tUFrWirKiKi_TAnsV...|Department Stores...|       Tucson|{8:0-23:0, 8:0-22...|      0|              Target|          22|  3.5|   AZ|
|         935 Race St|{null, null, u'no...|MTSW4McQd7CbVtyjq...|Restaurants, Food...| Philadelphia|{7:0-21:0, 7:0-20...|     

In [0]:
#Renaming some columns
business_sdf = business_sdf.withColumnRenamed('stars', 'business_stars')
business_sdf = business_sdf.withColumnRenamed('name', 'business_name')
business_sdf = business_sdf.withColumnRenamed('review_count', 'business_review_count')
business_sdf = business_sdf.withColumnRenamed('categories', 'business_categories')
business_sdf = business_sdf.withColumnRenamed('address', 'business_address')


In [0]:
business_sdf.count()

In [0]:
### Cleaning JSON File #2: yelp_academic_dataset_review.json ###

In [0]:
yelp_review_file = 's3a://yelp-reviews-jb/landing/yelp_academic_dataset_review.json'
reviews_sdf = spark.read.json(yelp_review_file)

In [0]:
reviews_sdf.count()

In [0]:
### Cleaning JSON File #3: yelp_academic_dataset_user.json ###

In [0]:
yelp_user_file = 's3a://yelp-reviews-jb/landing/yelp_academic_dataset_user.json'
users_sdf = spark.read.json(yelp_user_file)

In [0]:
# New dataframe with no null values

from pyspark.sql.functions import col

users_sdf = users_sdf.dropna()

for column in sdf5_cleaned.columns:
    null_count = sdf5_cleaned.filter(col(column).isNull()).count()
    print(f"Null count in {column}: {null_count}")

In [0]:
#Renaming columns so it begins with 'user_'
users_sdf = users_sdf.select([col(c).alias("user_"+c) for c in users_sdf.columns])
users_sdf = users_sdf.withColumnRenamed('user_user_id', 'user_id')

In [0]:
users_sdf.count()

In [0]:
# Join business file with review file
yelp_sdf = reviews_sdf.join(business_sdf, "business_id")

# Join business+reviews with users
yelp_sdf = yelp_sdf.join(users_sdf, "user_id")

In [0]:
# Dropping Two More Unnecessary Column
yelp_sdf = yelp_sdf.drop('attributes', 'hours')

In [0]:
# Checking columns in yelp_sdf
yelp_sdf.columns
yelp_sdf.dtypes

In [0]:
# Checking size of yelp_sdf
yelp_sdf.count()

In [0]:
# Setting File Output Path
yelp_parquet = 's3a://yelp-reviews-jb/raw/yelp_dataset.parquet'

# Writing to Output Path 
yelp_sdf.write.parquet(yelp_parquet, mode='overwrite')